# Loading Data

In [24]:
import pandas as pd

data1 = pd.read_csv( 'data/train.csv')

data2 = pd.read_csv( 'data/comments.csv')

data3 = pd.read_csv( 'data/ar_reviews_100k.csv')

data = pd.concat( ( data1, data2, data3)).reset_index(drop=True)

import sklearn
data = sklearn.utils.shuffle( data)

X = data[ 'comment']
y = data[ 'label']

In [25]:
print( "Number of Comments in data:", data.shape[0])

Number of Comments in data: 102984


# Defining Tokenisation Functions

In [4]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = set( stopwords.words('arabic'))

import string
punctuation = list( string.punctuation)
stop.update( punctuation)

import re
from clean import wordopt

def split_into_words(text):
    words = text.split()
    return words

def remove_punctuation(words):
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def remove_stopwords(words):
    stop_words = set(stopwords.words('arabic'))
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    return ' '.join(words)

def denoise_text(text):
    text = wordopt( text)
    words = split_into_words(text)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aminb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Tokenizing the data comments

In [5]:
vocabulary = data.apply( denoise_text)
data

0       انا اوصي من هذا المنبر الكل للتوجه الى مراكز ا...
1       \nهناك الكثير لا يفهم المقصود كورونا ليست صعبة...
2       الحمد لله ارقام مقبولة مقارنة بدول المنطقة لول...
3       انا شخصيا أؤيد ما فرضته السلطات من ضرورة الإدل...
4       \nنفس الشئ في مدينة برشيد مراكز التلقيح مغلقة ...
                              ...                        
1915    \nوالله حتى دمرونا وتعداو علينا شي ميقراش\nشي ...
1916                      لقاح موت و سم فيقو ا عباد الله 
1917    اللقاح غير آمن  ولا يمكن تجاهل عشرات شهادات ال...
1918    لنكن واقعيين…لا يجب فرظ التلقيح بالقوة  انا تل...
1919    \n#التنسيقية_الوطنية_للمغاربة_الرافضين_للتلقيح...
Name: comment, Length: 1919, dtype: object

In [6]:
vocabulary

0       [اوصي, منبر, توجه, مراكز, تلقيح, صدقوني, امر, ...
1       [كثير, يفهم, مقصود, كورونا, صعبة, شباب, لكنها,...
2       [حمد, لله, ارقام, مقبولة, مقارنة, بدول, منطقة,...
3       [شخصيا, اؤيد, فرضته, سلطات, ضرورة, إدلا, بجواز...
4       [مدينة, برشيد, مراكز, تلقيح, مغلقة, غاية, محاس...
                              ...                        
1915    [دمرونا, وتعداو, علينا, ميقراششي, ميخدمش, فلمي...
1916                    [لقاح, موت, سم, فيقو, عباد, الله]
1917    [لقاح, آمن, يمكن, تجاهل, عشرات, شهادات, اطبا, ...
1918    [لنكن, واقعيين, يجب, فرظ, تلقيح, بالقوة, تلقيت...
1919    [تنسيقية, وطنية, مغاربة, رافضين, تلقيح, إجباري...
Name: comment, Length: 1919, dtype: object

# Building Word2Vec Model

In [7]:
import gensim

w2v_model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

# Training Word2Vec Model

In [8]:
w2v_model.build_vocab( vocabulary, progress_per=1000)

In [9]:
w2v_model.train( vocabulary, total_examples=w2v_model.corpus_count, epochs=1000)

(38503596, 52006000)

# Saving the Word2Vec Model

In [10]:
w2v_model.save("models/word2vec-arabic-moroccen-dialect.model")

# Testing the WordVec Model

In [12]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load("models/word2vec-arabic.model")

w2v_model.wv.most_similar("خبز")

[('بلدك', 0.5379806160926819),
 ('يحمد', 0.5372437238693237),
 ('بالمال', 0.5169765949249268),
 ('باكملها', 0.5093464255332947),
 ('عقباه', 0.45860323309898376),
 ('يملك', 0.4556187689304352),
 ('مسؤولي', 0.4080964922904968),
 ('سائر', 0.405232310295105),
 ('مغفلين', 0.4018534719944),
 ('يهتم', 0.3988811671733856)]